In [0]:
%matplotlib inline

20183300 윤성빈


Spatial Transformer Networks Tutorial
=====================================
**Author**: `Ghassen HAMROUNI <https://github.com/GHamrouni>`_

.. figure:: /_static/img/stn/FSeq.png

In this tutorial, you will learn how to augment your network using
a visual attention mechanism called spatial transformer
networks. You can read more about the spatial transformer
networks in the `DeepMind paper <https://arxiv.org/abs/1506.02025>`__

Spatial transformer networks are a generalization of differentiable
attention to any spatial transformation. Spatial transformer networks
(STN for short) allow a neural network to learn how to perform spatial
transformations on the input image in order to enhance the geometric
invariance of the model.
For example, it can crop a region of interest, scale and correct
the orientation of an image. It can be a useful mechanism because CNNs
are not invariant to rotation and scale and more general affine
transformations.

One of the best things about STN is the ability to simply plug it into
any existing CNN with very little modification.



In [0]:
# License: BSD
# Author: Ghassen Hamrouni

from __future__ import print_function #파이선 2.에서 몇몇 기능을 파이썬 3와 같이 사용하게 해주고
import torch #파이토치 모듈을 사용하겠다고 선언
import torch.nn as nn #신경망 구조를 nn으로 사용하겠다고 선언
import torch.nn.functional as F #전환을 도와주는 패키지
import torch.optim as optim #최적화 알고리즘을 구현하는 패키지
import torchvision #인기있는 데이터셋을 가지고 있는 패키지
from torchvision import datasets, transforms #데이터셋과 변형패키지
import matplotlib.pyplot as plt #파이썬에서 데이터를 다양한 방법으로 시각화 시켜주는 패키지
import numpy as np #배열을 다루기 좋은 구조를 제공하는 패키지
 
plt.ion()   # interactive mode

Loading the data
----------------

In this post we experiment with the classic MNIST dataset. Using a
standard convolutional network augmented with a spatial transformer
network.



In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #GPU 상에서 tensor를 사용

# Training dataset
train_loader = torch.utils.data.DataLoader(                  #데이터 세트와 샘플러를 결합하고 주어진 데이터 세트에 대해 반복 가능을 제공함
    datasets.MNIST(root='.', train=True, download=True,      # root - 학습데이터나 테스트 데이터가 존재하는 루트 디렉토리, train- True라면 학습데이터를 만듦, downloda- True 라면 인터넷에서 데이터 셋을 받아와서 루트디렉토리에 넣음   
                   transform=transforms.Compose([            # transform- 필 이미지를 변환한다, transforms.Compose - 여러 변환을 구성한다. , transforms.ToTensor- 필 이미지나 넘파이 배열을 텐서로 변환한다.
                       transforms.ToTensor(),                # transforms.Normalize - 평균 및 표준 편차로 텐서 이미지를 표준화 한다. (평균, 표준 편차, inplace= False)
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4) #batch_size - 배치 당 로드할 샘플 수(기본값 = 1), shuffle- 매번 다시 섞을 거면 True로 해야함 (디폴트 = False), num_workers- 데이터 로드에 사용할 하위 프로세스의 수-- 데이터가 기본 프로세스에 로드 됨을 의미함.
# Test dataset
test_loader = torch.utils.data.DataLoader(                   # 위 데이터 셋은 학습을 위한 데이터 셑이고 현재 만드는 데이터 셑은 테스트를 위한 데이터이다
    datasets.MNIST(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True, num_workers=4)

0it [00:00, ?it/s]

9920512it [00:05, 1845100.80it/s]                             


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


32768it [00:00, 442519.24it/s]
  1%|          | 16384/1648877 [00:00<00:11, 140788.88it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:00, 7350322.68it/s]                           
8192it [00:00, 174112.65it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw
Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


Depicting spatial transformer networks
--------------------------------------

Spatial transformer networks boils down to three main components :

-  The localization network is a regular CNN which regresses the
   transformation parameters. The transformation is never learned
   explicitly from this dataset, instead the network learns automatically
   the spatial transformations that enhances the global accuracy.
-  The grid generator generates a grid of coordinates in the input
   image corresponding to each pixel from the output image.
-  The sampler uses the parameters of the transformation and applies
   it to the input image.

.. figure:: /_static/img/stn/stn-arch.png

.. Note::
   We need the latest version of PyTorch that contains
   affine_grid and grid_sample modules.




In [0]:
class Net(nn.Module):                        #Net 클래스 선언
    def __init__(self):
        super(Net, self).__init__()          #super- 자식 클래스에서 부모클래스의 내용을 사용하고 싶을경우 사용, 슈퍼클래스의 method를 호출하라는 의미인데, 이때 다수의 슈퍼 클래스가 존재시 클래스 호출 순서의 결정은 __mro__ 를 통해 결정된다. 
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #conv1d - 여러 입력면으로부터 구성된 입력 신호를 1차원 전환에 적용 (in_channels-입력 이미지의 채널 수, out_channels- 전환에 의해 제공된 채널의 수, kernel_size- 전환 커널의 크기, stride=1- 전환의 보폭, padding=0- 입력 양쪽에 추가된 제로 패딩의 수, dilation=1- 커널 요소 사이의 간격, groups=1- 입력과 출력 채널 사이에 차단된 연결 수, bias=True- True 면 학습 가는한 bias를 출력에 추가, padding_mode='zeros')
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)#conv2d - 여러 입력면으로부터 구성된 입력 신호를 2차원 전환에 적용 (위와 같음.)
        self.conv2_drop = nn.Dropout2d()             #전체 채널을 임의로 제로화(각 채널은 순방향 통화에서 독립적으로 제로화 됨)
        self.fc1 = nn.Linear(320, 50)                #들어오는 데이터에 선형 변환을 적용 ( in_features- 각 입력 샘플의 크기 , out_features- 각 출력 샘플의 크기 , bias = True)
        self.fc2 = nn.Linear(50, 10)                 

        # 공간 변형 위치 결정 네트워크
        self.localization = nn.Sequential(           #순차 컨테이너, 생성자에 전달된 순서대로 모듈이 추가됨.
            nn.Conv2d(1, 8, kernel_size=7),          #위의 conv2 내용과 같음
            nn.MaxPool2d(2, stride=2),               #여러 입력 평면으로 구성된 입력 신호에 2차원 최대 풀링을 적용함.(kernal size- 최대 크기의 윈도우, 윈도우의 보폭)
            nn.ReLU(True),                           #정류된 선형함수를 요소 단위로 적용
            nn.Conv2d(8, 10, kernel_size=5),         #위의 conv2 내용과 같음
            nn.MaxPool2d(2, stride=2),               
            nn.ReLU(True)                            
        )

        # 3 * 2 배열을 위한 회귀
        self.fc_loc = nn.Sequential(                 #순차 컨테이너, 생성자에 전달된 순서대로 모듈이 추가됨.
            nn.Linear(10 * 3 * 3, 32),               #선형 변환 적용
            nn.ReLU(True),                           #정류된 선형함수를 요소 단위로 적용 
            nn.Linear(32, 3 * 2)                     #선형 변환 적용
        )

        # 정체성 변형을 통한 weights와 bias 초기화
        self.fc_loc[2].weight.data.zero_()           #fc_loc의 2번째 데이터의 weight 데이터를 제로화
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))  #fc_loc의 2번째 데이터의 bias 데이터를 추가

    # 공간 변형 네트워크 forward 함수
    def stn(self, x):                      #stn 함수 선언
        xs = self.localization(x)          #위에 있는 self.localization 함수에 x 를 대입한 것을 xs 변수에 저장
        xs = xs.view(-1, 10 * 3 * 3)       #텐서의 사이즈나 모양을 변경하고 싶을 때 view 함수를 사용한다 xs의 사이즈를 (2, 10 * 3 * 3) 크기로 변환하였다.
        theta = self.fc_loc(xs)            #위에 있는 self.fc_loc 함수에 xs를 대입한 것을 theta 변수에 저장
        theta = theta.view(-1, 2, 3)       #theta의 사이즈를 (2, 2, 3) 크기로 변환하였다.

        grid = F.affine_grid(theta, x.size()) #functional.affine_grid(theta- 모양이 있는 아핀 행렬의 입력 배치 , size- 대상 출력의 이미지 크기 )- 아핀 매트릭스가 주어지면 2D 또는 3D 흐름 장 (샘플링 그리드)를 생성함.
        x = F.grid_sample(x, grid)            #functional.grid_sample(input- 입력의 모양, grid- 모양의 흐름장)- 인렵과 흐름장이 주어지면 값과 픽셀 위치를 사용하여 grid를 계산함.

        return x

    def forward(self, x):                  #forward 함수 선언
        # 입력 변환
        x = self.stn(x)                    # 입력값을 stn(x)로 변환

        # forwad 이동 실행
        x = F.relu(F.max_pool2d(self.conv1(x), 2))                  #self.conv1(x)의 윈도우에 2만큼의 보폭으로 최대 풀링을 적용함
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2)) #self.conv2(x)를 제로화 시키고 그 윈도우에 2만큼 최대 풀링을 적용함.
        x = x.view(-1, 320)                                         #x의 사이즈를 (-1, 320) 크기로 변환
        x = F.relu(self.fc1(x))                                     #x에 선형 변환을 적용(입력320, 출력50)
        x = F.dropout(x, training=self.training)                    #임의로 x의 일부를 제로화 한다.
        x = self.fc2(x)                                             #x에 선형 변환을 적용(입력50, 출력10)
        return F.log_softmax(x, dim=1)                              #x를 1차원에서 로그를 적용


model = Net().to(device)                   #model변수에 Net()함수 적용과 device에 캐스트                             

Training the model
------------------

Now, let's use the SGD algorithm to train the model. The network is
learning the classification task in a supervised way. In the same time
the model is learning STN automatically in an end-to-end fashion.



In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01)  #스토캐스틱 하강 그래디언트를 구하는 함수로서 (params-최적화를 위한 반복가능한 파라미터, lr=<required parameter>- 학습 속도)


def train(epoch):                                   #학습 함수 선언 (epoch-학습량)
    model.train()                                   #모듈을 학습 상태로 설정
    for batch_idx, (data, target) in enumerate(train_loader):       #train_loader에 있는 인덱스 값을 순서대로 batch_idx, data, target에 열거
        data, target = data.to(device), target.to(device)           #data 와 target 변수는 device에 캐스트하는 함수

        optimizer.zero_grad()                                       #모든 최적화된 그라디언트 제로화
        output = model(data)                                        #출렵값 적용
        loss = F.nll_loss(output, target)                           #F.nll_loss-음수 로그 우도 손실
        loss.backward()                                             #주어진 텐서의그라디언트 합계를 그래프로 계산
        optimizer.step()                                            #한 번의 최적화 단계 실행
        if batch_idx % 500 == 0:                                    #batch_idx가 500의 배수일 때
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(           #학습량과 데이타 길이, 데이터 셋 길이 등을 출력
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
#
# MNIST에서 STN의 성능을 측정하는 간단한 테스트 절차
#


def test():                           #테스트 함수 선언
    with torch.no_grad():             #그래디언트 계산 비활성화
        model.eval()                  #평가 모드로 변경
        test_loss = 0                 #틀린 값
        correct = 0                   #옳은 값
        for data, target in test_loader:        #테스트 과정
            data, target = data.to(device), target.to(device)
            output = model(data)

            # batch 손실 합계
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # 최대 로그 확률의 지수 구하기
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)         #합계를 데이터 셑 길이로 나눔
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'        #손실과, 정답, 데이터 셋 길이, 정답률 출력
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

Visualizing the STN results
---------------------------

Now, we will inspect the results of our learned visual attention
mechanism.

We define a small helper function in order to visualize the
transformations while training.



In [0]:
def convert_image_np(inp):                                  #넘파이 이미지로 변환하는 함수 선언
    """Convert a Tensor to numpy image."""                  #'텐서를 넘파이 이미지로 변환합니다.'
    inp = inp.numpy().transpose((1, 2, 0))                  #받은 변수를 넘파이로 변환하고 차원을 바꾼다.
    mean = np.array([0.485, 0.456, 0.406])                  #평균 배열을 생성한다.
    std = np.array([0.229, 0.224, 0.225])                   #표준편차 배열을 생성한다.
    inp = std * inp + mean                                  #이미지 넘파이 값 = 표준편차* 이미지넘파이 + 평균
    inp = np.clip(inp, 0, 1)                                #이미지 넘파이의 범위를 0~1로 제한
    return inp

# 학습 후에 공간 변형 레이어의 출력을 시각화하기 위해 STN을 사용하여 입력 이미지 배치와 해당 변형 배치를 시각화

def visualize_stn():
    with torch.no_grad():                                   #그래디언트 계산 비활성화
        # Get a batch of training data                      #트레이닝 데이터의 배치를 얻기
        data = next(iter(test_loader))[0].to(device)        #테스트 로더 데이터를 꺼내와 반환한 후 0번째 변수를 device에 전달

        input_tensor = data.cpu()                           #cpu 메모리에 복사본을 반환하는 함수
        transformed_input_tensor = model.stn(data).cpu()    #데이터를 그리드의 복사본을 cpu메모리에 반환하는 함수

        in_grid = convert_image_np(                         #data를 넘파이 이미지로 변환
            torchvision.utils.make_grid(input_tensor))     

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))  #데이터의 그리드를 넘파이 이미지로 변환

        # 결과를 나란히 표시
        f, axarr = plt.subplots(1, 2)                 #시각화 행 1개 열 2개
        axarr[0].imshow(in_grid)                      #data 이미지를 보여줌
        axarr[0].set_title('Dataset Images')          # 데이터 셋 이미지

        axarr[1].imshow(out_grid)                     #변환된 이미지를 보여줌
        axarr[1].set_title('Transformed Images')      #변환된 이미지


for epoch in range(1, 20 + 1):                        #20번 훈련과 테스트
    train(epoch)
    test()

# 일부 입력 배치에서 STN 변형 시각화
visualize_stn()            

plt.ioff()                            #대화식 모드 끄기
plt.show()                            #생성된 모든 figure 보여주기


#reference: https://pytorch.org/tutorials/intermediate/spatial_transformer_tutorial.html
            https://pytorch.org/docs/stable/index.html
            https://m.blog.naver.com/PostView.nhn?blogId=qbxlvnf11&logNo=221449297033&proxyReferer=https%3A%2F%2Fwww.google.com%2F
            https://kongdols-room.tistory.com/97
            https://pytorch.org/docs/stable/tensors.html?highlight=cpu#torch.Tensor.cpu

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309755
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.736031


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2645, Accuracy: 9218/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.654258
